# Линейная регрессия

## Щека Дмитрий, 24.М41

Я взял тот же датасет, что и для разведочного анализа, с монстрами из DND. Предсказывать в нем я буду рейтинг опасности (**cr**). Все преобразования я уже там сделал, поэтому со всеми изменениями датасет лежит в репозитории: dnd.csv

Я претендую на 7/10 баллов, за все обязательные плюс RFE (ну может еще +1, если засчитаете, что я гиперпараметры только для своей реализации подбирал, но не для Ridge)

### Описание датасета:
- **name** --- имя существа
- **cr** --- challange rating (целевой признак). Сложность столкновения с существом
- **type** --- тип существа (животное, дракон...)
- **ac** --- armor class (класс доспеха), как тяжело попасть по существу
- **hp** --- health points (очки здоровья)
- **align** --- говорит о стандартном поведении существа
- **legendary** --- является ли существо легендарным
- **source** --- из какой книги взято существо
- **str, dex, con, int, wis, cha** --- характеристики существа: сила, ловкость, телосложение, интеллект, мудрость, харизма

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# устанавливаем точность чисел с плавающей точкой
%precision %.4f

import warnings
warnings.filterwarnings('ignore')  # отключаем предупреждения

In [2]:
df = pd.read_csv('dnd.csv')
df.head()

,cr,type,size,ac,align,legendary,str,dex,con,int,wis,cha,hp_category
0,0.25,4,2,12,0,0,10.0,14.0,10.0,11.0,12.0,11.0,0
1,9.00,4,2,12,2,0,15.0,13.0,15.0,11.0,12.0,12.0,4
2,10.00,7,3,17,8,1,21.0,9.0,15.0,18.0,15.0,18.0,6
3,9.00,6,4,15,7,0,20.0,12.0,18.0,10.0,13.0,12.0,6
4,23.00,9,2,21,3,0,17.0,13.0,17.0,12.0,13.0,13.0,7


Сначала выберем признаки с помошью RFECV, потом разобьем на обучающую и тестовую выборку (возьмем побольше тестовую, потому что данных мало)

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.feature_selection import RFECV

y_train = df["cr"]
X_train = df.drop(columns=["cr"])

rfecv = RFECV(estimator=Ridge(), step=1, cv=5)
rfecv.fit(X_train, y_train)

selected_features = list(X_train.columns[rfecv.support_])

print("Optimal number of features: %d" % rfecv.n_features_)
print('Selected features: %s' % selected_features)

y = df["cr"]
X = df[selected_features]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

Optimal number of features: 9
Selected features: ['size', 'ac', 'align', 'legendary', 'str', 'con', 'wis', 'cha', 'hp_category']


Проведем масштабирование

In [4]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

#Со стандартным похуже результаты, я еще другой попробовал
#scaler = StandardScaler()  
scaler = MinMaxScaler(feature_range=(0, 1))
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Попробуем сначала обучить с помошью sklearn, и еще сравним, повлияло ли как-то масштабирование на результат (просто из интереса)

In [5]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

print("### Unscaled: ###")

model = Ridge()
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
y_train_pred = model.predict(X_train)

print("Test MSE = %.4f" % mean_squared_error(y_test, y_test_pred))
print("Train MSE = %.4f" % mean_squared_error(y_train, y_train_pred))
print("Test RMSE = %.4f" % mean_squared_error(y_test, y_test_pred, squared=False))
print("Train RMSE = %.4f" % mean_squared_error(y_train, y_train_pred, squared=False))
print("Test R2 = %.4f" % r2_score(y_test, y_test_pred))
print("Train R2 = %.4f" % r2_score(y_train, y_train_pred))

print('### Scaled: ###')

model = Ridge()
model.fit(X_train_scaled, y_train)
y_test_pred = model.predict(X_test_scaled)
y_train_pred = model.predict(X_train_scaled)

print("Test MSE = %.4f" % mean_squared_error(y_test, y_test_pred))
print("Train MSE = %.4f" % mean_squared_error(y_train, y_train_pred))
print("Test RMSE = %.4f" % mean_squared_error(y_test, y_test_pred, squared=False))
print("Train RMSE = %.4f" % mean_squared_error(y_train, y_train_pred, squared=False))
print("Test R2 = %.4f" % r2_score(y_test, y_test_pred))
print("Train R2 = %.4f" % r2_score(y_train, y_train_pred))

### Unscaled: ###
Test MSE = 6.4667
Train MSE = 7.6103
Test RMSE = 2.5430
Train RMSE = 2.7587
Test R2 = 0.8376
Train R2 = 0.8039
### Scaled: ###
Test MSE = 6.5332
Train MSE = 7.6738
Test RMSE = 2.5560
Train RMSE = 2.7702
Test R2 = 0.8360
Train R2 = 0.8022


## Реализация линейной регрессии

Реализовывать я буду как кастомный estimator для sklearn, чтобы с кросс-валидацией сразу нормально работало

In [6]:
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.metrics import mean_squared_error
from math import sqrt

class MyRegression(BaseEstimator, RegressorMixin):
    def __init__(self, alpha = 1.0, learning_multiplier = 1, tol = 0.001):
        self.alpha = alpha
        self.learning_multiplier = learning_multiplier
        self.tol = tol
    
    def fit(self, X, y):
        self.m, self.n = X.shape
        self.X = X
        self.y = y
        self.w = np.ones(self.n)
        self.b = 0
        
        y_pred = self.predict(self.X)
        err = mean_squared_error(self.y, y_pred)
        new_err = 0
        i = 1
        while (abs(new_err - err) > self.tol):
            err = new_err
            
            self.calc_weights(y_pred, i)
            
            y_pred = self.predict(self.X)
            new_err = mean_squared_error(self.y, y_pred)
            i += 1
        
        return self
    
    def predict(self, X):
        return X.dot(self.w) + self.b
    
    def calc_weights(self, y_pred, i):
        dW = (-(2 * (self.X.T).dot(self.y - y_pred)) + 2 * self.alpha * self.w) / self.m
        db = -2 * np.sum(self.y - y_pred) / self.m
        
        self.w = self.w - self.learning_multiplier * (1/sqrt(i)) * dW 
        self.b = self.b - self.learning_multiplier * (1/sqrt(i)) * db
        
        return self

In [7]:
model = MyRegression()
model.fit(X_train_scaled, y_train)
y_test_pred = model.predict(X_test_scaled)
y_train_pred = model.predict(X_train_scaled)

print("Test MSE = %.4f" % mean_squared_error(y_test, y_test_pred))
print("Train MSE = %.4f" % mean_squared_error(y_train, y_train_pred))
print("Test RMSE = %.4f" % mean_squared_error(y_test, y_test_pred, squared=False))
print("Train RMSE = %.4f" % mean_squared_error(y_train, y_train_pred, squared=False))
print("Test R2 = %.4f" % r2_score(y_test, y_test_pred))
print("Train R2 = %.4f" % r2_score(y_train, y_train_pred))

Test MSE = 6.9248
Train MSE = 8.0887
Test RMSE = 2.6315
Train RMSE = 2.8441
Test R2 = 0.8261
Train R2 = 0.7916


Получилось чуть-чуть хуже, чем с Ridge. Но я уверен, если поиграться с гиперпараметрами, можно сделать не хуже...

In [8]:
from sklearn.model_selection import GridSearchCV

alpha_grid = np.logspace(-5, 1, 10)
searcher = GridSearchCV(MyRegression(), [{"alpha": alpha_grid}], scoring="neg_root_mean_squared_error", cv=4)
searcher.fit(X_train_scaled, y_train)
best_alpha = searcher.best_params_["alpha"]
print("Best alpha for mine regression = %.5f" % best_alpha)

grid = np.logspace(0, 1.5, 10, base=2)
searcher = GridSearchCV(MyRegression(), [{"learning_multiplier": grid}], scoring="neg_root_mean_squared_error", cv=4)
searcher.fit(X_train_scaled, y_train)
best = searcher.best_params_["learning_multiplier"]
print("Best learning_multiplier for mine regression = %.4f" % best)

grid = np.logspace(-6, 0, 10)
searcher = GridSearchCV(MyRegression(), [{"tol": grid}], scoring="neg_root_mean_squared_error", cv=4)
searcher.fit(X_train_scaled, y_train)
best = searcher.best_params_["tol"]
print("Best tol for mine regression = %.6f" % best)

Best alpha for mine regression = 0.00001
Best learning_multiplier for mine regression = 2.2449
Best tol for mine regression = 0.000001


In [9]:
model = Ridge()
model.fit(X_train_scaled, y_train)
y_test_pred = model.predict(X_test_scaled)
y_train_pred = model.predict(X_train_scaled)
print("Ridge Test R2 = %.4f" % r2_score(y_test, y_test_pred))
print("Ridge Train R2 = %.4f" % r2_score(y_train, y_train_pred))

model = MyRegression(0.00001, 2.5198, 0.000001)
model.fit(X_train_scaled, y_train)
y_test_pred = model.predict(X_test_scaled)
y_train_pred = model.predict(X_train_scaled)
print("Mine Test R2 = %.4f" % r2_score(y_test, y_test_pred))
print("Mine Train R2 = %.4f" % r2_score(y_train, y_train_pred))

Ridge Test R2 = 0.8360
Ridge Train R2 = 0.8022
Mine Test R2 = 0.8368
Mine Train R2 = 0.8038


Вот, получилось не хуже, но без подбора гиперпараметров для Ridge. Но, пожалуй, оставлю так, чтобы было чем хвастаться...
Еще моя реализация сильно медленнее работает(

## Кросс-валидация

Данных как-то мало, так что я даже не 5 фолдов взял, а 4...

In [10]:
from sklearn.model_selection import cross_validate

scores = cross_validate(MyRegression(0.00001, 2.2449, 0.000001), X_train_scaled, y_train, cv=4, 
                        return_train_score=True, scoring=['r2', 'neg_root_mean_squared_error', 'neg_mean_squared_error'])


In [11]:
scores_df = pd.DataFrame(scores)
scores_df = scores_df.drop(["fit_time", "score_time"], axis=1)
scores_df = scores_df.apply(abs, axis=1)
scores_df.rename(columns=
          {'test_neg_root_mean_squared_error':'test_rmse', 
           'train_neg_root_mean_squared_error':'train_rmse', 
           'test_neg_mean_squared_error':'test_mse',
           'train_neg_mean_squared_error':'train_mse'}, inplace= True)
scores_df = scores_df.transpose()
scores_df['E'] = scores_df.apply(np.mean, axis=1)
scores_df['STD'] = scores_df.apply(np.std, axis=1)

In [12]:
scores_df

,0,1,2,3,E,STD
test_r2,0.813353,0.786290,0.768998,0.812265,0.795227,0.016653
train_r2,0.799764,0.808582,0.813854,0.799677,0.805469,0.005405
test_rmse,2.584580,2.848221,3.317518,2.504825,2.813786,0.283875
train_rmse,2.823328,2.731686,2.584033,2.848724,2.746943,0.092700
test_mse,6.680053,8.112360,11.005925,6.274149,8.018122,1.659381
train_mse,7.971179,7.462110,6.677224,8.115230,7.556436,0.503224


Понятно, что будет какой-то относительно большой разюрос будет присутствовать: данных мало, они разные и все такое. 

Также совершенно закономерно, что на валидационных кусках разброс выше, чем на тренировочных (более чем в три раза)

Еще понятно, что хороший результат на тренировочной части не гарантирует хорошего результата на валидационной (тут, например, на фолде с наилучшим результатом тренировки -- наихудший результат на валидации)

In [13]:
scores = cross_validate(Ridge(), X_train_scaled, y_train, cv=4, 
                        return_train_score=True, scoring=['r2', 'neg_root_mean_squared_error', 'neg_mean_squared_error'])
scores_df = pd.DataFrame(scores)
scores_df = scores_df.drop(["fit_time", "score_time"], axis=1)
scores_df = scores_df.apply(abs, axis=1)
scores_df.rename(columns=
          {'test_neg_root_mean_squared_error':'test_rmse', 
           'train_neg_root_mean_squared_error':'train_rmse', 
           'test_neg_mean_squared_error':'test_mse',
           'train_neg_mean_squared_error':'train_mse'}, inplace= True)
scores_df = scores_df.transpose()
scores_df['E'] = scores_df.apply(np.mean, axis=1)
scores_df['STD'] = scores_df.apply(np.std, axis=1)
scores_df

,0,1,2,3,E,STD
test_r2,0.823068,0.784113,0.762343,0.810036,0.794890,0.020971
train_r2,0.796440,0.806158,0.812180,0.797308,0.803021,0.005826
test_rmse,2.516418,2.862691,3.364967,2.519650,2.815932,0.310199
train_rmse,2.846671,2.748927,2.595624,2.865523,2.764186,0.095623
test_mse,6.332361,8.195000,11.323005,6.348637,8.049751,1.820907
train_mse,8.103534,7.556601,6.737262,8.211225,7.652155,0.522008
